In [1]:
import json
from tqdm import tqdm

In [2]:
train_path = 'train_dataset.txt'
dev_path = 'dev_dataset.txt'

In [3]:
train_text = ''
dev_text = ''
train_file = open(train_path, 'w')
dev_file = open(dev_path, 'w')
for i in tqdm(range(1,24274)):
    try:
        f = open("C:\\Users\\IvanH\\gaokao_mcq_answering\\train\\high\\" + str(i)+ ".txt", "r")
    except:
        continue
    train_text += json.loads(f.read())['article'] + '\n'
    f.close()
for i in tqdm(range(1,8286)):
    try:
        f = open("C:\\Users\\IvanH\\gaokao_mcq_answering\\train\\middle\\" + str(i)+ ".txt", "r")
    except:
        continue
    train_text += json.loads(f.read())['article'] + '\n'
    f.close()

for i in tqdm(range(63,24268)):
    try:
        f = open("C:\\Users\\IvanH\\gaokao_mcq_answering\\dev\\high\\" + str(i)+ ".txt", "r")
    except:
        continue
    dev_text += json.loads(f.read())['article'] + '\n'
    f.close()
for i in tqdm(range(13,8281)):
    try:
        f = open("C:\\Users\\IvanH\\gaokao_mcq_answering\\dev\\middle\\" + str(i)+ ".txt", "r")
    except:
        continue
    dev_text += json.loads(f.read())['article'] + '\n'
    f.close()
train_file.write(train_text)
dev_file.write(dev_text)

100%|██████████| 8268/8268 [00:01<00:00, 6540.70it/s]


2180411

In [4]:
from transformers import GPT2TokenizerFast,GPT2LMHeadModel
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

Training codes are partially adapted from https://github.com/philschmid/fine-tune-GPT-2/blob/master/Fine_tune_a_non_English_GPT_2_Model_with_Huggingface.ipynb. 

In [5]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(dev_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=32)
        #   block_size=128)
     
    dev_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=dev_path,
          block_size=32) 
        #   block_size=128)  
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,dev_dataset,data_collator

train_dataset,dev_dataset,data_collator = load_dataset(train_path,dev_path,tokenizer)

C:\Users\IvanH\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
from transformers import Trainer, TrainingArguments
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('gpt2')


training_args = TrainingArguments(
    output_dir="./output", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

In [7]:
trainer.train()
trainer.save_model()

100%|██████████| 849/849 [04:41<00:00,  3.01it/s]
{'train_runtime': 281.9421, 'train_samples_per_second': 3.011, 'epoch': 3.0}


In [8]:
import pandas as pd 
data_df = pd.read_csv('data.csv')
data_df.head()

,question,A,B,C,D,Answer
0,I can't remember _____________ made the teach...,that it was what,what it was that,what was it that,that was it what,B
1,"He let out an ______________ cry, ""we've won!""",excited,exciting,excite,excites,A
2,Is football John's favourite sport? Yes. ____...,Near to,Except,Beside,Next to,D
3,Do you think regular exercise ___________ goo...,benefit from,reach for,make for,go for,C
4,Have you applied ___________ Mr Black _______...,for; to,with; for,with; about,to; for,D


In [9]:
import torch
import sys
import numpy as np
 
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
# Load pre-trained model (weights)
with torch.no_grad():
        model = GPT2LMHeadModel.from_pretrained('output')
        model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
 
def score(sentence):
    tokenize_input = tokenizer.encode(sentence)
    tensor_input = torch.tensor([tokenize_input])
    loss=model(tensor_input, labels=tensor_input)[0]
    return np.exp(loss.detach().numpy())

In [10]:
import re
prediction_list=[]
answer_list= []
for i in tqdm(range(len(data_df))):
    row = data_df.iloc[i]
    question_original = row['question'].strip()
    choice_list = ['A','B','C','D']
    score_list = []
    sentence_list = []

    if ';' in row['A']:
        try:
            for choice in choice_list:
                choice_split = [i.strip() for i in row[choice].split(';')]
                if len(choice_split) == 2:
                    question_text = re.sub('\_+',' '+choice_split[0]+' ', question_original,1)
                    question_text = re.sub('\_+',' '+choice_split[1]+' ', question_text,1)
                if len(choice_split) == 3:
                    question_text = re.sub('\_+',' '+choice_split[0]+' ', question_original,1)
                    question_text = re.sub('\_+',' '+choice_split[1]+' ', question_text,1)
                    question_text = re.sub('\_+',' '+choice_split[2]+' ', question_text,1)
                sentence_list.append(question_text)
        except:
                print(question_original,question_text,choice_split)
                continue
    else:
        sentence_list = [re.sub('\_+',' '+row[choice].strip()+' ', question_original) for choice in choice_list]

    if ('/' in row['A']+row['B']+row['C']+row['D']):
        sentence_list = [re.sub('/','',sentence) for sentence in sentence_list]

    sentence_list = [re.sub(' +',' ', sentence) for sentence in sentence_list]

    for sentence in sentence_list:
        score_list.append(score(sentence))
    final_choice = choice_list[np.argmin(np.array(score_list))]
    answer_list.append(final_choice)
    if final_choice == row['Answer']:
        prediction_list.append(1)
    else:
        prediction_list.append(0)

100%|██████████| 1183/1183 [06:36<00:00,  2.98it/s]


In [11]:
np.array(prediction_list).sum()/len(prediction_list)

0.6830092983939138